# Feature Engineering on Sports Dataset

## Overview
This notebook performs feature engineering on an American football dataset. The goal is to create new meaningful features that can enhance predictive analysis.

- **Dataset:** Historical game scores
- **Objective:** Create additional features to extract insights and improve future modeling efforts.

---

## Data Loading & Initial Inspection
We start by loading the dataset and examining its structure.

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('spreadspoke_scores.csv')


In [16]:
print(df.shape)
print(df.columns)
df.head()


(14073, 17)
Index(['schedule_date', 'schedule_season', 'schedule_week', 'schedule_playoff',
       'team_home', 'score_home', 'score_away', 'team_away',
       'team_favorite_id', 'spread_favorite', 'over_under_line', 'stadium',
       'stadium_neutral', 'weather_temperature', 'weather_wind_mph',
       'weather_humidity', 'weather_detail'],
      dtype='object')


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45,7,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14,19,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN


In [17]:
df.describe()

,schedule_season,score_home,score_away,spread_favorite,over_under_line,weather_temperature,weather_wind_mph,weather_humidity
count,14073.000000,14073.000000,14073.000000,11594.000000,11522.000000,12537.000000,12521.000000,8488.000000
mean,1997.167626,22.506360,19.839906,-5.361135,42.331470,59.017628,7.562974,67.190386
std,16.699765,10.517811,10.141783,3.430043,4.796631,15.525791,5.729531,15.814698
min,1966.000000,0.000000,0.000000,-26.500000,28.000000,-6.000000,0.000000,4.000000
25%,1983.000000,15.000000,13.000000,-7.000000,39.000000,48.000000,1.000000,57.000000
50%,1998.000000,22.000000,20.000000,-4.500000,42.000000,62.000000,8.000000,69.000000
75%,2012.000000,30.000000,27.000000,-3.000000,45.500000,72.000000,11.000000,79.000000
max,2024.000000,72.000000,62.000000,0.000000,63.500000,97.000000,40.000000,100.000000


## Feature Engineering
In this section, we create new features based on existing data. We will generate:
- `score_diff`: The difference between the home and away team scores.
- `team_won`: Categorical feature indicating whether the home team, away team, or neither won the match.
- `team_home_utd`, `team_away_utd`: Creating a mapping for up to date team names from historical team names
- `score_total`: Total score of the game
- `over_or_under`: Over Under mapping
- `df_regular_season_records`: Create new df with regular season records
- `df_playoff_records`: Create new df with playoff records
- `mean_scores_df`: Create new df with mean scores in regular season and playoff
---

In [18]:
# Create a new column for the score difference
#df['score_diff'] = df['score_home'] - df['score_away']
df.insert(7, 'score_diff', df['score_home'] - df['score_away'])


In [19]:
# Function to determine which team won
def diff_based_value(score_diff):
    if   score_diff > 0:
        return 'home'
    elif score_diff < 0:
        return 'away'
    else:
        return 'tie';

df['team_won'] = df['score_diff'].apply(diff_based_value)
team_won = df.pop('team_won')

df.insert(8, 'team_won', team_won)

In [20]:
df['team_won'].value_counts()

,count
team_won,
home,8025
away,5957
tie,91


In [21]:
TEAM_NAME_MAPPING = {
    # Cardinals franchise
    "Arizona Cardinals": "Arizona Cardinals",
    "Phoenix Cardinals": "Arizona Cardinals",
    "St. Louis Cardinals": "Arizona Cardinals",

    # Falcons
    "Atlanta Falcons": "Atlanta Falcons",

    # Ravens
    "Baltimore Ravens": "Baltimore Ravens",

    # Bills
    "Buffalo Bills": "Buffalo Bills",

    # Panthers
    "Carolina Panthers": "Carolina Panthers",

    # Bears
    "Chicago Bears": "Chicago Bears",

    # Bengals
    "Cincinnati Bengals": "Cincinnati Bengals",

    # Browns
    "Cleveland Browns": "Cleveland Browns",

    # Cowboys
    "Dallas Cowboys": "Dallas Cowboys",

    # Broncos
    "Denver Broncos": "Denver Broncos",

    # Lions
    "Detroit Lions": "Detroit Lions",

    # Packers
    "Green Bay Packers": "Green Bay Packers",

    # Texans
    "Houston Texans": "Houston Texans",

    # Colts (Baltimore -> Indianapolis)
    "Indianapolis Colts": "Indianapolis Colts",
    "Baltimore Colts": "Indianapolis Colts",  # pre-1984

    # Jaguars
    "Jacksonville Jaguars": "Jacksonville Jaguars",

    # Chiefs
    "Kansas City Chiefs": "Kansas City Chiefs",

    # Chargers (San Diego -> Los Angeles)
    "Los Angeles Chargers": "Los Angeles Chargers",
    "San Diego Chargers": "Los Angeles Chargers",  # 1961–2016

    # Rams (Los Angeles -> St. Louis -> Los Angeles)
    "Los Angeles Rams": "Los Angeles Rams",
    "St. Louis Rams": "Los Angeles Rams",  # 1995–2015

    # Dolphins
    "Miami Dolphins": "Miami Dolphins",

    # Vikings
    "Minnesota Vikings": "Minnesota Vikings",

    # Patriots (Boston -> New England)
    "New England Patriots": "New England Patriots",
    "Boston Patriots": "New England Patriots",  # pre-1971

    # Saints
    "New Orleans Saints": "New Orleans Saints",

    # Giants
    "New York Giants": "New York Giants",

    # Jets
    "New York Jets": "New York Jets",

    # Eagles
    "Philadelphia Eagles": "Philadelphia Eagles",

    # Steelers
    "Pittsburgh Steelers": "Pittsburgh Steelers",

    # Raiders (Oakland -> Los Angeles -> Oakland -> Las Vegas)
    "Las Vegas Raiders": "Las Vegas Raiders",
    "Los Angeles Raiders": "Las Vegas Raiders",  # 1982–1994
    "Oakland Raiders": "Las Vegas Raiders",      # 1960–1981 & 1995–2019

    # 49ers
    "San Francisco 49ers": "San Francisco 49ers",

    # Seahawks
    "Seattle Seahawks": "Seattle Seahawks",

    # Buccaneers
    "Tampa Bay Buccaneers": "Tampa Bay Buccaneers",

    # Titans (Houston Oilers -> Tennessee Oilers -> Tennessee Titans)
    "Tennessee Titans": "Tennessee Titans",
    "Tennessee Oilers": "Tennessee Titans",  # 1997–1998
    "Houston Oilers": "Tennessee Titans",    # pre-1997

    # Washington (Redskins -> Football Team -> Commanders)
    "Washington Commanders": "Washington Commanders",
    "Washington Football Team": "Washington Commanders",  # 2020–2021
    "Washington Redskins": "Washington Commanders"        # pre-2020
}

historical_teams = [
    "St. Louis Cardinals",
    "Boston Patriots",
    "Washington Redskins",
    "Houston Oilers",
    "San Diego Chargers",
    "New York Giants"
]

normalized_teams = [TEAM_NAME_MAPPING.get(team, team) for team in historical_teams]
print(normalized_teams)


['Arizona Cardinals', 'New England Patriots', 'Washington Commanders', 'Tennessee Titans', 'Los Angeles Chargers', 'New York Giants']


In [22]:
# Create new, standardised columns
df["team_home_utd"] = df["team_home"].map(TEAM_NAME_MAPPING)
df["team_away_utd"] = df["team_away"].map(TEAM_NAME_MAPPING)

# Insert them immediately after the originals
df.insert(
    # Position: one after "team_home"
    df.columns.get_loc("team_home") + 1,
    "team_home_utd",
    df.pop("team_home_utd")  # pop temporarily removes & returns the col
)

df.insert(
    # Position: one after "team_away" (note: this is after reordering from the previous insert)
    df.columns.get_loc("team_away") + 1,
    "team_away_utd",
    df.pop("team_away_utd")
)


In [23]:
df.isnull().sum()

,0
schedule_date,0
schedule_season,0
schedule_week,0
schedule_playoff,0
team_home,0
team_home_utd,0
score_home,0
score_away,0
score_diff,0
team_won,0


In [24]:
# Add Game total score column
df.insert(9, 'score_total', df['score_home'] + df['score_away'])


In [25]:
# Create the O/U data
over_or_under = np.where(
    (df['score_total'].notna()) & (df['over_under_line'].notna()),
    np.where(
        df['score_total'] > df['over_under_line'],
        'O',
        np.where(
            df['score_total'] < df['over_under_line'],
            'U',
            'Push'
        )
    ),
    np.nan
)

#over_or_under = df.pop('over_or_under')
df.insert(16, 'over_or_under', over_or_under)

In [26]:
df.head()

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,team_home_utd,score_home,score_away,score_diff,score_total,...,team_favorite_id,spread_favorite,over_under_line,over_or_under,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,Miami Dolphins,14,23,-9,37,...,NaN,NaN,NaN,nan,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,Tennessee Titans,45,7,38,52,...,NaN,NaN,NaN,nan,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,Los Angeles Chargers,27,7,20,34,...,NaN,NaN,NaN,nan,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,Miami Dolphins,14,19,-5,33,...,NaN,NaN,NaN,nan,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,Green Bay Packers,24,3,21,27,...,NaN,NaN,NaN,nan,Lambeau Field,False,64.0,8.0,62.0,NaN


In [ ]:
#df.to_csv("enhanced_scores1.csv", index=False)